### Read RSS data into dataframe

In [ ]:
import psycopg2
database = "rssfeed"
hostname="rssfeed.cjgj2uy1bapa.us-east-1.rds.amazonaws.com"
port="5432" 
userid="postgres"
passwrd=""
conn_string = "host="+hostname+" port="+port+" dbname="+database+" user="+userid+" password="+passwrd
conn = psycopg2.connect(conn_string)
conn.autocommit=True
cursor = conn.cursor()
sqlSelect = "SELECT * FROM rss_entities_with_pub";
cursor.execute(sqlSelect);
rows = cursor.fetchall();

In [ ]:
## Publisher url codes
import pandas as pd
publishers = pd.read_json('data/rss_feed.json', typ='series')
publishers = publishers.to_dict()
pub = {v: k for k, v in publishers.items()}

In [ ]:
rssdf = pd.DataFrame(rows)
rssdf.columns = ['publisher_code','entity','publish_time']
rssdf["publisher"] = rssdf["publisher_code"].map(pub)  
rssdf.head()

,publisher_code,entity,publish_time,publisher
0,5871,Auckland | Coco,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed
1,5871,Wozniacki,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed
2,5871,Tennis Threads Magazine,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed
3,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...
4,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...


### Create a 24-hour time bucket

In [ ]:
rssdf['24hour_bucket'] = pd.to_datetime(rssdf['publish_time'],errors='coerce',format = '%Y-%m-%dT%H:%M:%S+00:00',\
                                        infer_datetime_format = True, cache = True)





In [ ]:
from datetime import dateime
print(rssdf['publish_time'][100])
datetime.fromisoformat(rssdf['publish_time'][100])

2020-01-23T23:02:17+00:00


datetime.datetime(2020, 1, 23, 23, 2, 17, tzinfo=datetime.timezone.utc)

In [ ]:
rssdf.head()

,publisher_code,entity,publish_time,publisher,24hour_bucket
0,5871,Auckland | Coco,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52
1,5871,Wozniacki,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52
2,5871,Tennis Threads Magazine,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52
3,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...,2019-12-31 08:00:00
4,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...,2019-12-31 08:00:00


In [ ]:
rssdf['24hour_bucket'][0]

Timestamp('2020-01-07 06:23:52')

In [ ]:
rssdf['dates']=rssdf['24hour_bucket'].dt.date

In [ ]:
rssdf.head()


,publisher_code,entity,publish_time,publisher,24hour_bucket,dates
0,5871,Auckland | Coco,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52,2020-01-07
1,5871,Wozniacki,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52,2020-01-07
2,5871,Tennis Threads Magazine,2020-01-07T06:23:52+00:00,http://tennisthreads.net/feed,2020-01-07 06:23:52,2020-01-07
3,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...,2019-12-31 08:00:00,2019-12-31
4,13317,Nigerian,2019-12-31T08:00:00+00:00,https://news.google.com/rss?hl=en-US&gl=US&cei...,2019-12-31 08:00:00,2019-12-31


### Group by time bucket, get a list of entities for each time bucket

In [ ]:
newdf = pd.DataFrame(rssdf.groupby(['dates'])['entity'].apply(", ".join)).reset_index().sort_values(by=['dates'])

In [ ]:
newdf['dates2'] = pd.to_datetime(newdf['dates'])

In [1]:
import datetime
newdf = newdf[newdf['dates2']<datetime.date(2020,2,23)]  

NameError: name 'newdf' is not defined

In [ ]:
newdf.head()

,dates,entity,dates2
0,1970-01-01,London,1970-01-01
1,1970-07-02,"Kevin Pollak's, Kevin Pollak's",1970-07-02
2,1993-01-03,Christians,1993-01-03
3,1993-01-10,"James English, Bible",1993-01-10
4,1993-01-24,"Solomon, Christians, Greg",1993-01-24


### Find entity overlaps between every pair of consecutive time buckets

In [ ]:
def common(a,b): 
    #c = [value for value in a if value in b] 
    c = set(a).intersection(b)
    return ','.join(c)

common(['hello','abc','perry'],['hello','perry','goodmorning'])

'hello,perry'

In [ ]:
newdf

,dates,entity,dates2
0,1970-01-01,London,1970-01-01
1,1970-07-02,"Kevin Pollak's, Kevin Pollak's",1970-07-02
2,1993-01-03,Christians,1993-01-03
3,1993-01-10,"James English, Bible",1993-01-10
4,1993-01-24,"Solomon, Christians, Greg",1993-01-24
...,...,...,...
5720,2020-02-18,"ABR, YEE (YEE), YEE (CURRENCY, YEE, US, DigiFi...",2020-02-18
5721,2020-02-19,"Kate, Property Auction, Jonie, US, Jonathan, D...",2020-02-19
5722,2020-02-20,"HEDS, EQ-5D-5L, HTA, ReQol, Valuing Health:, D...",2020-02-20
5723,2020-02-21,"The Real Flower Petal Confetti Company, Garden...",2020-02-21


In [ ]:
# Compared two consecutive days, but be careful about the date.
# one week slicing window? ==> calculate simliarty, dominate entity?
import numpy as np
newdf['overlaps'] = ""
def findoverlaps(newdf):
    for i in range(len(newdf)-1):
        left = list(newdf['entity'][i].split(","))
        right = list(newdf['entity'][i+1].split(","))
        newdf['overlaps'].values[i+1] = common(left,right)
        #newdf.at[i+1, 'overlaps'] = common(left, right)
        
    return(newdf)

overlaps = findoverlaps(newdf) 

In [ ]:
overlaps

,dates,entity,dates2,overlaps
0,1970-01-01,London,1970-01-01,
1,1970-07-02,"Kevin Pollak's, Kevin Pollak's",1970-07-02,
2,1993-01-03,Christians,1993-01-03,
3,1993-01-10,"James English, Bible",1993-01-10,
4,1993-01-24,"Solomon, Christians, Greg",1993-01-24,
...,...,...,...,...
5720,2020-02-18,"ABR, YEE (YEE), YEE (CURRENCY, YEE, US, DigiFi...",2020-02-18,", FCoin, Us, American Dream, Harry Maguire, Wa..."
5721,2020-02-19,"Kate, Property Auction, Jonie, US, Jonathan, D...",2020-02-19,", OANDA, FCoin, Us, P Chidambaram, Nashvilles,..."
5722,2020-02-20,"HEDS, EQ-5D-5L, HTA, ReQol, Valuing Health:, D...",2020-02-20,", OANDA, FCoin, Us, P Chidambaram, Nashvilles,..."
5723,2020-02-21,"The Real Flower Petal Confetti Company, Garden...",2020-02-21,", OANDA, Us, Balls Mahoney, Lacey Evans, Toron..."


### Find percentage of overlaps 

In [ ]:
def percentoverlap(a,b): # params are lists
    setA = set(a)
    setB = set(b)
    overlap = setA & setB
    universe = setA | setB
    result1 = float(len(overlap)) / len(setA) * 100
    result2 = float(len(overlap)) / len(setB) * 100
    result3 = float(len(overlap)) / len(universe) * 100
    return(result1,result2,result3)

percentoverlap(['hello','perry'],['hello','goodmorning'])

(50.0, 50.0, 33.33333333333333)

In [ ]:
newdf['left_percent'] = 0
newdf['right_percent'] = 0
newdf['union_percent'] = 0
def detectevents(newdf):
    for i in range(len(newdf)-1):
        if newdf['overlaps'].values[i] == "":
            pass
        else:
            a = list(newdf['overlaps'][i].split(","))
            b = list(newdf['overlaps'][i+1].split(","))
            newdf['left_percent'].values[i+1] = percentoverlap(a,b)[0]
            newdf['right_percent'].values[i+1] = percentoverlap(a,b)[1]
            newdf['union_percent'].values[i+1] = percentoverlap(a,b)[2]
    return(newdf)
detectevents(newdf)  

,dates,entity,dates2,overlaps,left_percent,right_percet,union_percent,right_percent
0,1970-01-01,London,1970-01-01,,0,0,0,0
1,1970-07-02,"Kevin Pollak's, Kevin Pollak's",1970-07-02,,0,0,0,0
2,1993-01-03,Christians,1993-01-03,,0,0,0,0
3,1993-01-10,"James English, Bible",1993-01-10,,0,0,0,0
4,1993-01-24,"Solomon, Christians, Greg",1993-01-24,,0,0,0,0
...,...,...,...,...,...,...,...,...
5720,2020-02-18,"ABR, YEE (YEE), YEE (CURRENCY, YEE, US, DigiFi...",2020-02-18,", FCoin, Us, American Dream, Harry Maguire, Wa...",64,0,38,48
5721,2020-02-19,"Kate, Property Auction, Jonie, US, Jonathan, D...",2020-02-19,", OANDA, FCoin, Us, P Chidambaram, Nashvilles,...",64,0,44,59
5722,2020-02-20,"HEDS, EQ-5D-5L, HTA, ReQol, Valuing Health:, D...",2020-02-20,", OANDA, FCoin, Us, P Chidambaram, Nashvilles,...",61,0,44,61
5723,2020-02-21,"The Real Flower Petal Confetti Company, Garden...",2020-02-21,", OANDA, Us, Balls Mahoney, Lacey Evans, Toron...",60,0,44,61


### Filter by 70% overlap and higher

In [ ]:
newdf[newdf['right_percent'] > 70]

,dates,entity,dates2,overlaps,left_percent,right_percet,union_percent,right_percent
91,1997-02-16,"Bible, Christians",1997-02-16,Christians,100,0,100,100
114,1998-02-22,"Christians, Greg, Titanic",1998-02-22,Greg,100,0,100,100
115,1998-03-01,"Old Testament, God, Israel, Greg, New Testament",1998-03-01,Greg,100,0,100,100
123,1998-05-24,"Eating Jesus/Bread of Heaven, Greg, Christian",1998-05-24,Greg,100,0,100,100
124,1998-05-31,"Pakistan, India, Rest of the World, Greg, Chri...",1998-05-31,Greg,100,0,100,100
...,...,...,...,...,...,...,...,...
5662,2019-12-22,"Stella, Trump, Everlane, Jaana, Thanksgiving, ...",2019-12-22,", Northampton, Strictly Come Dancing, Californ...",43,0,37,72
5697,2020-01-26,"Boris Johnson, Tory, West Midlands, HS2, Conse...",2020-01-26,", Lucy Fallon, RT.com, Jason Brown, Us, Bike L...",41,0,36,73
5700,2020-01-29,"Kirstie, Phil, Love, Rushden, Susan, John, Kir...",2020-01-29,", ITV News, Lapland, Smackdown Results, Whiteh...",17,0,16,78
5704,2020-02-02,"San Francisco, Chiefs on Super Bowl, EPL OK, R...",2020-02-02,", RT.com, Zimbabwe, Boxing News, Frank B. Half...",51,0,44,77
